# Fake News Prediction

## Preparation

In [2]:
!pip install pyspellchecker

In [205]:
#Library

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import string
import re

from collections import defaultdict
from collections import Counter

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.util import ngrams
stop = list(stopwords.words("english"))


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, log_loss,confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score, f1_score
from spellchecker import SpellChecker

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nathasya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [206]:
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

In [207]:
fake["type"] = 1
true["type"] = 0

In [208]:
lst = []
for text in true["text"]:
  temp = re.sub(r"^.*Reuters+\W+[-]+\W", r"", text)
  lst.append(temp)

In [209]:
true["text"] = lst

In [210]:
df = pd.concat([fake,true], axis = 0).reset_index(drop = True)

In [211]:
df.head()

,title,text,subject,date,type
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [212]:
df.tail()

,title,text,subject,date,type
44893,'Fully committed' NATO backs new U.S. approach...,NATO allies on Tuesday welcomed President Dona...,worldnews,"August 22, 2017",0
44894,LexisNexis withdrew two products from Chinese ...,"LexisNexis, a provider of legal, regulatory an...",worldnews,"August 22, 2017",0
44895,Minsk cultural hub becomes haven from authorities,In the shadow of disused Soviet-era factories ...,worldnews,"August 22, 2017",0
44896,Vatican upbeat on possibility of Pope Francis ...,Vatican Secretary of State Cardinal Pietro Par...,worldnews,"August 22, 2017",0
44897,Indonesia to buy $1.14 billion worth of Russia...,Indonesia will buy 11 Sukhoi fighter jets wort...,worldnews,"August 22, 2017",0


## Preprocessing



### Cleaning

In [213]:
def remove_url(text):
  url = re.compile(r"https?://\S+|www\.\S+")
  return url.sub(r"", text)

In [214]:
def remove_html(text):
  html = re.compile(r"<.*?>")
  return html.sub(r"", text)

In [215]:
def remove_emoji(text):
    emoji = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji.sub(r'', text)

In [216]:
def remove_punct(text):
  punct = re.compile(r'[!"#$%&\\\'()*+,-./:;<=>?@[\\\]^_`{|}~]')
  return punct.sub("", text)

In [217]:
def spell_checking(text):

  spell = SpellChecker()
  correct = []

  miss_spell = spell.unknown(text.split())

  for word in text.split():
    if word in miss_spell:
      correct.append(spell.correction(word))
    else:
      correct.append(word)
  
  return " ".join(correct)

In [218]:
def remove_stopword(text):
  temp = [word for word in text if word not in stop]
  return temp

In [219]:
def remove_single_alphabet(text):
  temp = [word for word in text if word not in string.ascii_lowercase]
  return temp

In [220]:
# def lower(text):
#     temp = [word.lowe() for word in text]
#     return temp
    

In [221]:
# def cleaning_text(text):
#   text = text.apply(lambda x : remove_url)
#   text = text.apply(lambda x : remove_html)
#   text = text.apply(lambda x : remove_emoji)
#   text = text.apply(lambda x : lower)
#   text = text.apply(lambda x : remove_punct)
#   # text = text.apply(lambda x : spell_checking(x))
#   return text

In [222]:
def cleaning_text(text):
  text = text.apply(lambda x : remove_url(x))
  text = text.apply(lambda x : remove_html(x))
  text = text.apply(lambda x : remove_emoji(x))
  text = text.apply(lambda x : x.lower())
  text = text.apply(lambda x : remove_punct(x))
  # text = text.apply(lambda x : spell_checking(x))
  return text 

In [223]:
def list_to_string(text):
  return " ".join(text)

In [224]:
df["text"] = cleaning_text(df["text"])
df["text"] = df["text"].apply(lambda x : x.lower())
df["text"] = df["text"].apply(lambda x : x.split())
df["text"] = df["text"].apply(remove_stopword)
df["text"] = df["text"].apply(remove_single_alphabet)
df["text"] = df["text"].apply(list_to_string)

In [225]:
df["text"]

0        donald trump wish americans happy new year lea...
1        house intelligence committee chairman devin nu...
2        friday revealed former milwaukee sheriff david...
3        christmas day donald trump announced would bac...
4        pope francis used annual christmas day message...
                               ...                        
44893    nato allies tuesday welcomed president donald ...
44894    lexisnexis provider legal regulatory business ...
44895    shadow disused sovietera factories minsk stree...
44896    vatican secretary state cardinal pietro paroli...
44897    indonesia buy 11 sukhoi fighter jets worth 114...
Name: text, Length: 44898, dtype: object

# Normalization

In [226]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nathasya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Stemming

In [227]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
from nltk.stem.porter import *

def stem(text):
#     token_words = word_tokenize(sentence)
#     token_words
#     stem_sentence={}
    stem_result = []
#           temp = [word for word in text if word not in stop]
#   return temp
    stemmer = PorterStemmer()
    
    for word in text:
        stem_result.append(stemmer.stem(word))
    return stem_result

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nathasya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Lemmatization

In [228]:
from nltk.stem import WordNetLemmatizer

def lemma(text):

    lemmatizer = WordNetLemmatizer()
#     token_words = word_tokenize(sentence)
#     token_words
    lem_result=[]
    
    for word in text:
        lem_result.append(lemmatizer.lemmatize(word))
    return lem_result

In [229]:
# df["text"] = cleaning_text(df["text"])
# # df["text"] = df["text"].apply(lambda x : x.lower())
# # df["text"] = df["text"].apply(lambda x : x.split())
# # df["text"] = df["text"].apply(remove_stopword)
# df["text"] = df["text"].apply(remove_single_alphabet)
# df["text"] = df["text"].apply(stem)
# df["text"] = df["text"].apply(lemma)
# df["text"] = df["text"].apply(list_to_string)

In [230]:
df["text"] = cleaning_text(df["text"])
df["text"] = df["text"].apply(lambda x : x.lower())
df["text"] = df["text"].apply(lambda x : x.split())
df["text"] = df["text"].apply(remove_stopword)
df["text"] = df["text"].apply(remove_single_alphabet)
df["text"] = df["text"].apply(stem)
df["text"] = df["text"].apply(lemma)
df["text"] = df["text"].apply(list_to_string)

In [231]:
df["text"]

0        donald trump wish american happi new year leav...
1        hous intellig committe chairman devin nune go ...
2        friday reveal former milwauke sheriff david cl...
3        christma day donald trump announc would back w...
4        pope franci use annual christma day messag reb...
                               ...                        
44893    nato alli tuesday welcom presid donald trump d...
44894    lexisnexi provid legal regulatori busi inform ...
44895    shadow disus sovietera factori minsk street li...
44896    vatican secretari state cardin pietro parolin ...
44897    indonesia buy 11 sukhoi fighter jet worth 114 ...
Name: text, Length: 44898, dtype: object

### Splitting Train & Test

In [232]:
# Splitting
from sklearn.model_selection import train_test_split

X = df["text"]
y = df["type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18)

## Feature Extraction



### Bag Of Word

In [233]:
# Bag of Word

vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000)

train_features = vectorizer.fit_transform(X_train)
train_features = train_features.toarray()

test_features = vectorizer.transform(X_test)
test_features = test_features.toarray()

### GloVe Vectorization

In [234]:
from tqdm import tqdm
from nltk.tokenize import word_tokenize

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nathasya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [235]:
# ps = PorterStemmer() 

def create_corpus(df):
  corpus = []

  for text in tqdm(df["text"]):
    words = [word.lower() for word in word_tokenize(text) if ((word.isalpha()) & (word not in stop))]
    corpus.append(words)
  
  return corpus

In [236]:
corpus = create_corpus(df)

100%|███████████████████████████████████████████████████████████████████████████| 44898/44898 [02:40<00:00, 279.39it/s]


In [241]:
!kaggle datasets download -d rtatman/glove-global-vectors-for-word-representation


  0%|          | 0.00/458M [00:00<?, ?B/s]
  0%|          | 1.00M/458M [00:00<02:19, 3.44MB/s]
  0%|          | 2.00M/458M [00:00<02:16, 3.51MB/s]
  1%|          | 3.00M/458M [00:00<02:08, 3.72MB/s]
  1%|          | 4.00M/458M [00:01<02:13, 3.57MB/s]
  1%|1         | 5.00M/458M [00:01<02:14, 3.53MB/s]
  1%|1         | 6.00M/458M [00:02<02:55, 2.71MB/s]
  2%|1         | 7.00M/458M [00:02<02:59, 2.63MB/s]
  2%|1         | 8.00M/458M [00:02<02:58, 2.64MB/s]
  2%|1         | 9.00M/458M [00:03<04:13, 1.86MB/s]
  2%|2         | 10.0M/458M [00:04<04:03, 1.93MB/s]
  2%|2         | 11.0M/458M [00:04<03:32, 2.20MB/s]
  3%|2         | 12.0M/458M [00:04<02:58, 2.62MB/s]
  3%|2         | 13.0M/458M [00:05<02:49, 2.75MB/s]
  3%|3         | 14.0M/458M [00:05<02:37, 2.95MB/s]
  3%|3         | 15.0M/458M [00:05<02:29, 3.11MB/s]
  3%|3         | 16.0M/458M [00:06<02:20, 3.29MB/s]
  4%|3         | 17.0M/458M [00:06<02:20, 3.28MB/s]
  4%|3         | 18.0M/458M [00:06<02:22, 3.23MB/s]
  4%|4         | 19.

In [242]:
from zipfile import ZipFile

file_name = "glove-global-vectors-for-word-representation.zip"

with ZipFile(file_name, "r") as files:
  files.extractall()
  print("Done")


 42%|####1     | 192M/458M [01:10<01:22, 3.40MB/s]
 42%|####2     | 193M/458M [01:10<01:21, 3.41MB/s]
 42%|####2     | 194M/458M [01:11<01:17, 3.57MB/s]
 43%|####2     | 195M/458M [01:11<01:19, 3.47MB/s]
 43%|####2     | 196M/458M [01:11<01:21, 3.38MB/s]
 43%|####3     | 197M/458M [01:12<01:20, 3.42MB/s]
 43%|####3     | 198M/458M [01:12<01:19, 3.45MB/s]
 43%|####3     | 199M/458M [01:12<01:27, 3.11MB/s]
 44%|####3     | 200M/458M [01:13<01:27, 3.10MB/s]
 44%|####3     | 201M/458M [01:13<01:25, 3.15MB/s]
 44%|####4     | 202M/458M [01:13<01:28, 3.05MB/s]
 44%|####4     | 203M/458M [01:14<01:27, 3.05MB/s]
 45%|####4     | 204M/458M [01:14<01:22, 3.22MB/s]
 45%|####4     | 205M/458M [01:14<01:22, 3.22MB/s]
 45%|####4     | 206M/458M [01:15<01:20, 3.30MB/s]
 45%|####5     | 207M/458M [01:15<01:17, 3.40MB/s]
 45%|####5     | 208M/458M [01:15<01:17, 3.39MB/s]
 46%|####5     | 209M/458M [01:15<01:20, 3.24MB/s]
 46%|####5     | 210M/458M [01:16<01:14, 3.51MB/s]
 46%|####6     | 211M/458M [01

Done


In [248]:
embedding_dict = {}

file = open("glove.6B.100d.txt", encoding="utf8")
for line in file:
    values = line.split()
    words = values[0]
    vectors = np.asarray(values[1:], "float32")
    embedding_dict[words] = vectors

f.close()

In [252]:
!pip install virtualenv

  Created wheel for distlib: filename=distlib-0.3.0-py3-none-any.whl size=340432 sha256=2468e27bb85958d01bdbff43e2c22121b95b1235fd8db6fe63b871952c81d1ff
  Stored in directory: c:\users\nathasya\appdata\local\pip\cache\wheels\eb\4e\d2\a903d4184fb49e4ac06474d65715b129aee13d69f7d227e78e
Successfully built distlib


In [255]:
!virtualenv -p python3.6 Python_3_6

RuntimeError: failed to find interpreter for Builtin discover of python_spec='python3.6'


In [254]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


ImportError: Could not find the DLL(s) 'msvcp140_1.dll'. TensorFlow requires that these DLLs be installed in a directory that is named in your %PATH% environment variable. You may install these DLLs by downloading "Microsoft C++ Redistributable for Visual Studio 2015, 2017 and 2019" for your platform from this URL: https://support.microsoft.com/help/2977003/the-latest-supported-visual-c-downloads

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequence = tokenizer.texts_to_sequences(corpus)

text_pad = pad_sequences(sequence, maxlen = 50, truncating = "post", padding = "post")

In [ ]:
word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_mat = np.zeros((num_words, 100))

# print(word_index)

for word, idx in word_index.items():
  if idx > num_words:
    continue
  
  embedding_vec = embedding_dict.get(word)
  if embedding_vec is not None:
    embedding_mat[idx] = embedding_vec

## Modelling



### Random Forest



In [ ]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 100)
rf = rf.fit(train_features, y_train)
rf_pred = rf.predict(test_features)

In [ ]:
akurasi = accuracy_score(y_test, rf_pred)
presisi = precision_score(y_test, rf_pred)
recall = recall_score(y_test,rf_pred)

print("Akurasi : {}".format(akurasi))
print("Precision : {}".format(presisi))
print("Recall : {}".format(recall))

### Naive Bayes



In [ ]:
# Naive Bayes

from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb = nb.fit(train_features, y_train)
nb_pred = nb.predict(test_features)


In [ ]:
akurasi = accuracy_score(y_test, nb_pred)
presisi = precision_score(y_test, nb_pred)
recall = recall_score(y_test,nb_pred)

print("Akurasi : {}".format(akurasi))
print("Precision : {}".format(presisi))
print("Recall : {}".format(recall))

### Logistic Regression

In [ ]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression

reglog = LogisticRegression()
reglog = reglog.fit(train_features, y_train)
reglog_pred = reglog.predict(test_features)

In [ ]:
akurasi = accuracy_score(y_test, reglog_pred)
presisi = precision_score(y_test, reglog_pred)
recall = recall_score(y_test,reglog_pred)

print("Akurasi : {}".format(akurasi))
print("Precision : {}".format(presisi))
print("Recall : {}".format(recall))

### Light GBM

In [ ]:
!pip install lightgbm

In [ ]:
#LigthGBM

import lightgbm as lgb

lgbm = lgb.LGBMClassifier()
lgbm = lgbm.fit(train_features, y_train)
lgbm_pred = lgbm.predict(test_features)

In [ ]:
akurasi = accuracy_score(y_test, lgbm_pred)
presisi = precision_score(y_test, lgbm_pred)
recall = recall_score(y_test,lgbm_pred)

print("Akurasi : {}".format(akurasi))
print("Precision : {}".format(presisi))
print("Recall : {}".format(recall))

### Neural Network

In [ ]:
!pip install constants

In [ ]:
from tensorflow.python.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.python.keras.initializers import Constant
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras import Sequential

In [ ]:
# Model Architecture
model = Sequential()


embedding = Embedding(num_words, 100,
                      embeddings_initializer = Constant(embedding_mat), 
                      input_length = 50, trainable = False)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))

model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics = ["accuracy"])

model.summary()

In [ ]:
X = text_pad
y = df["type"]

In [ ]:
# Splitting Data

X_train, X_test, y_train, y_test = train_test_split(text_pad, y, test_size = 0.15)

print("Dimension of X_train : {}".format(X_train.shape))
print("Dimension of X_test : {}".format(X_test.shape))

In [ ]:
first = model.fit(x = X_train, y = y_train, batch_size = 4, epochs = 5, validation_data = (X_test, y_test), verbose = 2)

In [ ]:
nn_pred = model.predict(X_test)

In [ ]:
akurasi = accuracy_score(y_test, nn_pred.round())
presisi = precision_score(y_test, nn_pred.round())
recall = recall_score(y_test, nn_pred.round())

print("Akurasi : {}".format(akurasi))
print("Precision : {}".format(presisi))
print("Recall : {}".format(recall))